In [ ]:
import numpy as np
from astropy.io import fits
import healpy as hp
import pandas as pd
from desiutil.plots import plot_sky_binned
import matplotlib.pyplot as plt
import pickle




### Retrieving the relevant CCD Data

In [ ]:
decamCCD = fits.open('../bricks_data/ccds-annotated-decam-dr9.fits')
#mosaicCCD = fits.open('../bricks_data/ccds-annotated-mosaic-dr9.fits')
#bassCCD = fits.open('../bricks_data/ccds-annotated-90prime-dr9.fits')
print(decamCCD[1].columns)


In [ ]:
dataDecam = decamCCD[1].data
#dataMosaic = mosaicCCD[1].data
#dataBass = bassCCD[1].data

In [ ]:
ra0 = dataDecam.field('ra0')
dec0 = dataDecam.field('dec0')
ra1 = dataDecam.field('ra1')
dec1 = dataDecam.field('dec1')
ra2 = dataDecam.field('ra2')
dec2 = dataDecam.field('dec2')
ra3 = dataDecam.field('ra3')
dec3 = dataDecam.field('dec3')

In [ ]:
df = pd.read_csv('../bricks_data/galaxy_catalogue_sample.csv')

ra = df["RA"].to_numpy(copy=True)
dec = df["DEC"].to_numpy(copy=True)

raDec = np.stack((ra,dec), axis=1)

raDec.shape

### Defining Healpy Utilities

In [ ]:
def raDec2thetaPhi(ra, dec):
    return (0.5 * np.pi - np.deg2rad(dec)), (np.deg2rad(ra))

NSIDE = 256
NPIX = hp.nside2npix(NSIDE)


In [ ]:
ax = plot_sky_binned(ra, dec, plot_type='healpix', max_bin_area=1, verbose=True)


### First Step: Identify which CCD is associated to which pixel

In [ ]:
pixel2ccd_dict = {}
count_ex = 0

# In this function, I am first converting every CCD into a given polygon using its corner coordinates
# Afterwards, I am using the ang2vec function and the hp.query_polygon function, to identify every pixel that features the given CCD
# Then, I am creating a dictionary that lists all CCDs for a given pixel

for no in range(len(ra0)):#range(len(ra0)):
    x_coord_polygon = np.array([ra0[no],ra1[no], ra2[no], ra3[no]])
    y_coord_polygon = np.array([dec0[no],dec1[no], dec2[no], dec3[no]])
    theta, phi = raDec2thetaPhi(x_coord_polygon, y_coord_polygon)

    ccd_vertices_3d = hp.ang2vec(theta=theta, phi=phi)

    # Now, Im getting all healpy pixels that a given CCD covers
    try:
        pixels_for_ccd = hp.query_polygon(nside=NSIDE, vertices=ccd_vertices_3d, inclusive=True)
    except:
        count_ex += 1
        continue
    # This will give me the mapping CCD --> Pixel
    # Now, im reverting the mapping
    # For every pixel that a given CCD cuts, enter the CCD into the pixel dictionary
    # Mapping: Pixel --> CCDs
    for pixel in pixels_for_ccd:
        if pixel not in pixel2ccd_dict:
            pixel2ccd_dict[pixel] = []
        pixel2ccd_dict[pixel].append(no)

    if no % 40000 == 0:
        print(no, count_ex)

# Find out what happens to 2467 CCDs where query polygon throws unknown exception

In [ ]:
#Exporting to Pickle, since this process only needs to be run once
#The filename number (e.g.256) indicates the NSIDE of the Healpy pixels used to generate the pixalisation

with open('../bricks_data/brickpixel2ccd_256.pickle', 'wb') as f:
    pickle.dump(pixel2ccd_dict, f)
    f.close()


# with open('pixel2ccd_256.pickle') as f:
    #data = pickle.load(f)




#### Plotting the Boundaries of the different CCDs that are associated to an arbitrary pixel


In [ ]:
pixel_number = 620311
print(pixel2ccd_dict.keys())
print(len(pixel2ccd_dict.keys()))

for i, keys in enumerate(pixel2ccd_dict.keys()):
    break
    ccd = pixel2ccd_dict[keys]
    print("Number of CCDs that are cutting pixel number ", keys, ":", len(ccd))
    for cc in ccd:
    #coord = [[x0,y0], [x1,y1], [x2,y2], [x3,y3], [x0,y0]]
        xs = [ra0[cc],ra1[cc], ra2[cc], ra3[cc],ra0[cc] ]
        ys = [dec0[cc],dec1[cc], dec2[cc], dec3[cc],dec0[cc]]
        plt.plot(xs,ys)

    if i > 3:
        break
plt.show()

In [ ]:
print(NPIX)


